In [1]:
%config InlineBackend.figure_formats = ['svg']

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
from tqdm import tqdm_notebook as tqdm
import pickle
import os
from typing import List, Dict

In [2]:
os.chdir("/Users/edwardgan/Documents/Projects/sketchstore/python")

In [4]:
import sketch.compress_freq
import sketch.sketch_frequent
import sketch.compress_dyadic
import sketch.sketch_gen
import storyboard.board_gen
import cube_board
import cube_rquery

In [6]:
def get_error_file(
    data_name: str, split_strategy: str,
    board_size: int, sketch_name: str,
    bias: bool, workload_p: float
) -> str:
    sketch_file = cube_board.get_file_name(data_name, split_strategy, board_size, sketch_name, bias)
    error_file = cube_rquery.transform_sketch_to_error_file(sketch_file, workload_p)
    return error_file

In [39]:
data_name = "synthf@4"
board_size = 2048
workload_p = .2
board_params = [
        ("pps", "weighted@20", True),
        ("random_sample", "uniform", False),
        ("random_sample", "prop", False),
        ("truncation", "uniform", False),
        ("cms_min", "uniform", False),

        ("pps", "uniform", True),
        ("pps", "weighted@20", False),
        ("random_sample", "weighted@20", True),
]

e_fnames = [
    get_error_file(
        data_name, 
        split_strategy=split_strategy,
        board_size=board_size, 
        sketch_name=sketch_name,
        bias=bias,
        workload_p=workload_p, 
    )
    for sketch_name,split_strategy,bias in board_params
]
e_dfs = [pd.read_csv(f) for f in e_fnames]
e_combined = pd.concat(e_dfs, axis=0)
e_combined["max_n"] = e_combined["max"] / e_combined["total"]
eg1 = e_combined.groupby(["sketch", "_split_strategy", "_bias_opt"]).aggregate({
    "max_n": ["mean", "std", "count"],
})
eg2 = e_combined.groupby(["sketch", "_split_strategy", "_bias_opt", "query_len"]).aggregate({
    "max_n": ["mean", "std", "count"],
})

In [40]:
eg1

max_n                
                                             mean       std count
sketch        _split_strategy _bias_opt                          
cms_min       uniform         False      0.018966  0.000025  2000
pps           uniform         True       0.010795  0.006137  2000
              weighted@20     False      0.011275  0.012693  2000
                              True       0.006032  0.007126  2000
random_sample prop            False      0.021547  0.034182  2000
              uniform         False      0.026342  0.021494  2000
              weighted@20     True       0.018295  0.014210  2000
truncation    uniform         False      0.008690  0.000283  2000

In [41]:
eg2

max_n                
                                                       mean       std count
sketch        _split_strategy _bias_opt query_len                          
cms_min       uniform         False     0          0.018965  0.000000   786
                                        1          0.018965  0.000016   838
                                        2          0.018968  0.000035   317
                                        3          0.018985  0.000091    53
                                        4          0.019036  0.000191     6
pps           uniform         True      0          0.006589  0.000000   786
                                        1          0.010677  0.002530   838
                                        2          0.017815  0.005903   317
                                        3          0.029080  0.008891    53
                                        4          0.045729  0.006727     6
              weighted@20     False     0          0.004265  0.000000   786
                                        1          0.010480  0.005119   838
                                        2          0.022725  0.011970   317
                                        3          0.048494  0.027676    53
                                        4          0.106869  0.034770     6
                              True      0          0.001898  0.000000   786
                                        1          0.005964  0.003220   838
                                        2          0.011800  0.006617   317
                                        3          0.028411  0.015200    53
                                        4          0.054638  0.021438     6
random_sample prop            False     0          0.006850  0.000000   786
                                        1          0.019588  0.013359   838
                                        2          0.044100  0.043645   317
                                        3          0.113791  0.092628    53
                                        4          0.213894  0.161865     6
              uniform         False     0          0.013858  0.000000   786
                                        1          0.026981  0.013795   838
                                        2          0.043944  0.025444   317
                                        3          0.081582  0.030821    53
                                        4          0.154525  0.076003     6
              weighted@20     True      0          0.010837  0.000000   786
                                        1          0.016935  0.004912   838
                                        2          0.031909  0.017176   317
                                        3          0.058444  0.026380    53
                                        4          0.111529  0.049884     6
truncation    uniform         False     0          0.008661  0.000000   786
                                        1          0.008644  0.000154   838
                                        2          0.008772  0.000493   317
                                        3          0.009258  0.000763    53
                                        4          0.009619  0.001091     6